In [1]:
import re
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import pandas as pd
from typing import Dict, List, get_type_hints
from IPython.display import display
# ! pip install spacy
# python -m spacy download en_core_web_sm

In [2]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

In [3]:
df = pd.read_csv('生成所有matched句子/Dofloo_report_match_sent.csv')
print('shape',df.shape)
df.head()

shape (34, 4)


,report,baseline,sentence number,sentence
0,Dofloo-MalwareMustDie,/etc,38,1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n.text:0x0a7...
1,Dofloo-MalwareMustDie,/etc/.,38,1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n.text:0x0a7...
2,Dofloo-MalwareMustDie,/etc,40,".text:0x0a840 mov r1, #0x42\n.text:0x0a844 ..."
3,Dofloo-MalwareMustDie,/etc/.,40,".text:0x0a840 mov r1, #0x42\n.text:0x0a844 ..."
4,Dofloo-MalwareMustDie,nul,40,".text:0x0a840 mov r1, #0x42\n.text:0x0a844 ..."


In [4]:
df.loc[0, 'sentence']

"1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n.text:0x0a760  stmfd   sp!, {r4-r8,lr}\n.text:0x0a764  sub  sp, sp, #0x208\n.text:0x0a768  add  r7, sp, #0x108\n.text:0x0a76c  mov  r4, r0\n.text:0x0a770  mov  r8, r1\n.text:0x0a774  mov  r0, r7\n.text:0x0a778  mov  r1, #0x100\n.text:0x0a77c  bl   sub_0x026fb0\n.text:0x0a780  ldr  r0,  <-- chattr -i /etc/crontab\n.text:0x0a784  bl   sub_0x0e3e0\n.text:0x0a788  ldr  r0, <-- chmod +w /etc/crontab\n\nadding the autostart entry in it:\n\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n.text:0x0a7dc ; xref: sub_0x0a760\n.text:0x0a7dc  ldr  r1, <-- sed -i '/%s/d' /etc/crontab\n.text:0x0a7e0  mov  r2, r5\n.text:0x0a7e4  mov  r0, r6\n   :\n.text:0x0a800  ldr  r1, <-- echo '*/1 * * * * root %s/%s %s' >> /etc/crontab\n.text:0x0a804  str  r8, [sp,#0x108+var_108]\n.text:0x0a808  bl   sub_0x0182dc\n.text:0x0a80c  mov  r0, r6\n.text:0x0a810  bl   sub_0x0e3e0\n\ncreate the file: /etc/.mysys, which later on found as the self copy attempt."

### 客製化 tokenizer (Ver1)
1. 避免分割單字中的特殊字元

In [3]:
def select_target_sentence(ST: str, paragraph: str) -> str:
    '''若 input text 有多行，回傳第一行含有 ST 的句子'''
    if paragraph.find('\n') == -1:
        return paragraph
    for sent in paragraph.split('\n'):
        if sent.find(ST) != -1:
            return sent
    return None

In [3]:
# https://stackoverflow.com/questions/66636097/prevent-spacy-tokenizer-from-splitting-on-specific-character
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER, HYPHENS
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

nlp = spacy.load("en_core_web_sm")
text = "Get 10ct/liter off when using our App"
# Modify tokenizer infix patterns
infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        #r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
        r"(?<=[{a}0-9])[:<>=](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer
doc = nlp(text)
print([t.text for t in doc])
for token in doc:
    print('{:12}{:10}{:5}'.format(token.text, token.pos_, token.tag_))

['Get', '10ct/liter', 'off', 'when', 'using', 'our', 'App']
Get         VERB      VB   
10ct/liter  NUM       CD   
off         ADP       RP   
when        SCONJ     WRB  
using       VERB      VBG  
our         PRON      PRP$ 
App         PROPN     NNP  


In [7]:
lenx = len(df)
ST_list = []
Path("./images").mkdir(parents=True, exist_ok=True)
for no in range(lenx):
    ST = df.loc[no, 'baseline']
    target_text = select_target_sentence(ST, df.loc[no, 'sentence'])
    sentence_nlp = nlp(target_text)
    svg = displacy.render(sentence_nlp, style="dep", jupyter=False)
    output_path = Path(f"./images/no{no}_dependency_plot.svg") # 檔名不能用 Spacial token 因為含有 /
    ST_list.append(f'{no:<2} {ST:<25} {target_text}')
    # output_path.parent.mkdir(parents=True, exist_ok=True)
    with output_path.open("w", encoding="utf-8") as f:
        f.write(svg)
with open('./images/ST_list.txt', 'w', encoding="utf-8") as f:
    for id,metadata in enumerate(ST_list):
        f.write(f'{metadata}\n')

In [8]:
no = 6
ST = df.loc[no, 'baseline']
target_text = select_target_sentence(ST, df.loc[no, 'sentence'])
print('Special token:',ST,'|',target_text)

Special token: 48080 | the malware also connects to 23.224.59.34:48080 to send and receive remote shell commands from the attacker.


In [9]:
doc = nlp(target_text)
for token in doc:
    # print(token.text, token.tag_, token.head.text, token.dep_)
    print(f'{token.text: <12} {token.tag_:<4} {token.head.text:<10} {token.dep_}')

the          DT   malware    det
malware      NN   connects   nsubj
also         RB   connects   advmod
connects     VBZ  connects   ROOT
to           IN   connects   prep
23.224.59.34:48080 CD   to         pobj
to           TO   send       aux
send         VB   connects   xcomp
and          CC   send       cc
receive      VB   send       conj
remote       JJ   commands   amod
shell        NN   commands   compound
commands     NNS  receive    dobj
from         IN   receive    prep
the          DT   attacker   det
attacker     NN   from       pobj
.            .    connects   punct


In [59]:
token_id = -1
for i,token in enumerate(doc):
    if token.text.find(ST) >= 0:
        token_id = i
print(token_id, doc[token_id])

20 /etc/rc.d/rc.local


In [61]:
print([token.text for token in doc[token_id].rights])
list(doc[token_id].subtree)

[',']


[/etc/rc.d/rc.local, ,]

In [44]:
# displacy.serve(doc, style='dep')
displacy.render(doc, style='dep', jupyter=True)

In [20]:
# 斷句
about_doc = nlp(target_text)
sentences = list(about_doc.sents)
len(sentences)

for sentence in sentences:
    print('#',sentence)

# .text:0x0a780  ldr  r0,  <-- chattr -i /etc/crontab


In [11]:
target_text = "the virtual file /proc/cpuinfo is opened and read in words"
doc = nlp(target_text)
print(f"{'#token.text': <14} {'tag_':<4} {'head.text':<10} {'dep_'}")
for token in doc:
    # print(token.text, token.tag_, token.head.text, token.dep_)
    print(f'{token.text: <14} {token.tag_:<4} {token.head.text:<10} {token.dep_}')

#token.text    tag_ head.text  dep_
the            DT   file       det
virtual        JJ   file       amod
file           NN   opened     nsubjpass
/proc/cpuinfo  :    file       punct
is             VBZ  opened     auxpass
opened         VBN  opened     ROOT
and            CC   opened     cc
read           VBN  opened     conj
in             IN   read       prep
words          NNS  in         pobj


### 產圖片的 code

In [6]:
# target_text = """The malware first checks whether it’s running from the locations /usr/bin/, /bin/, or /tmp/."""
target_text = "The main method then branches to function named get_executable_name which reads the symlink /proc/self/exe via readlink(..)."
doc = nlp(target_text)
displacy.render(doc, style='dep', jupyter=True)

# 存圖檔
# Path("./images").mkdir(parents=True, exist_ok=True)
no = 999
# file_name = 'thesis_path_spacy'
file_name = 'thesis_path_spacy'
svg = displacy.render(doc, style="dep", jupyter=False)
output_path = Path(f"./images/no{no}_{file_name}.png") # 檔名不能用 Spacial token 因為含有 /
with output_path.open("w", encoding="utf-8") as f:
    f.write(svg)

In [12]:
spacy.explain('nmod')

'modifier of nominal'

In [11]:
print(f"{'#token.text': <14} {'tag_':<4} {'head.text':<10} {'dep_'}")
for token in doc:
    # print(token.text, token.tag_, token.head.text, token.dep_)
    print(f'{token.text: <14} {token.tag_:<4} {token.head.text:<10} {token.dep_}')

#token.text    tag_ head.text  dep_
hostos         NNS  Distribution compound
	              _SP  hostos     dep
Distribution   NNP  speed      nmod
	              _SP  Distribution dep
Extracted      VBN  Distribution acl
from           IN   Extracted  prep
/etc/issue     NN   from       pobj
or             CC   Distribution cc
/etc/redhat    NN   release    compound
-              HYPH release    punct
release        NN   Distribution conj
hostcpu        NN   Clock      compound
	              _SP  hostcpu    dep
Clock          NNP  speed      compound
speed          NN   speed      ROOT
	              _SP  speed      dep
/proc/cpuinfo  SYM  speed      punct


### Find verb fucntion
傳入一個句子和目標字，鎖定句子中的目標字，找出作用在目標字上的動詞。

In [5]:
def find_verb_of_vocab(sentence: str, target_word: str, lemma=True):
    'Pass a sentence and a target_word, return the verb operates on the target_word or None. Lemmatization applied at default.'
    doc = nlp(sentence)
    # find the target_word element (excat match)
    target_elem = None
    for i,token in enumerate(doc):
        if token.text == target_word:
            target_elem = token
    if target_elem is None:
        return None
    # find the verb parent of the target_word element
    token = target_elem
    while token.head != token:
        token = token.head
        # verb found
        if token.tag_.startswith('VB'):
            if lemma:
                return token.lemma_
            return token.text
    return None

# test 1, <connects, 23.224.59.34:48080>
sentence = 'the malware also connects to 23.224.59.34:48080 to send and receive remote shell commands'
target_word = '23.224.59.34:48080'
verb = find_verb_of_vocab(sentence, target_word)
print(f'test1: {verb}')

# test 2, <opened, /proc/cpuinfo>
sentence = 'the virtual file /proc/cpuinfo is opened and read in words'
target_word = '/proc/cpuinfo'
verb = find_verb_of_vocab(sentence, target_word)
print(f'test2: {verb}')

# test 3, <hardcode, /proc/cpuinfo>
sentence = "table 2: data collected by the malware and sent back to the c2 server url key	description	comment hostip	ip	hardcoded to 127.0.0.1 softtype		hardcoded to “linux” pscaddr	mac address	 hostname	machine name	 hosttar	username	possibly “host target” hostos	distribution	extracted from /etc/issue or /etc/redhat-release hostcpu	clock speed	/proc/cpuinfo hostmem	amount of memory	/proc/meminfo hostpack		hardcoded to “linux” lkmtag	is rootkit enabled	 kernel	kernel version	extracted from unamefigure 12 shows the communication between redxor and the c2."
target_word = '/proc/cpuinfo'
verb = find_verb_of_vocab(sentence, target_word)
print(f'test3: {verb}')

# test 4
sentence = "Trend Micro researchers also discovered that the latest variant of the AESDDoS bot can modify files i.e., /etc/rc.local and /etc/rc.d/rc.local, as an autostart technique by appending the {malware path}/{malware file name} reboot command."
target_word = '/etc/rc.d/rc.local'
verb = find_verb_of_vocab(sentence, target_word)
print(f'test4: {verb}')

test1: connect
test2: open
test3: hardcode
test4: modify


In [30]:
sentence = """Get the magic string value from the file /var/run/gcc.pid
Read the following from its configuration file:
md5 - checksum of file …
"""
target_word = '/var/run/gcc.pid'
verb = find_verb_of_vocab(sentence, target_word)
print(f'test3: {verb}')

test3: read


In [36]:
sentence = """Related URLs:
hxxp://62[.]171[.]160[.]189/linux_arm
hxxp://62[.]171[.]160[.]189/11/123.sh
XORDDoS and other malware variants found through the same URL
"""
target_word = 'hxxp://62[.]171[.]160[.]189/linux_arm'
verb = find_verb_of_vocab(sentence, target_word)
print(f'test3: {verb}')


test3: find


In [10]:
# case: regex 會抓到 '61.147.91.53.' 多一個點，導致找不到人和動詞。
sentence = """We pass in the first parameter in the register ‘r0’ from the location 0xC1FC8 which has the value of 61.147.91.53."""
target_word = '61.147.91.53'
verb = find_verb_of_vocab(sentence, target_word)
print(f'test3: {verb}')

doc = nlp(sentence)
svg = displacy.render(doc, style="dep", jupyter=True)

test3: have


- 若直接用原句下去做 nlp() 會產生被斷句的情形，讓後方 10 個物件沒有跟前句相連。
- 若把這些物件改成 dir_1 dir_2 dir_3 dir_4 dir_5 dir_6 字串，則 dir_1 會被誤判成動詞，傻眼。
- 若把這些物件改成 dir_proc_stat dir_proc_meminfo 等保有字面意義的字串，才能找到 gather 動詞。

In [10]:

sentence = """This Backdoor gathers the following data: CPU information Memory statistics IP address of infected machine 
Reads the following information from /proc: /proc/stat /proc/meminfo /proc/cpuinfo /proc/net/dev /proc/self/exe /proc/self/maps /proc/sys/vm/overcommit_memory /proc/sys/kernel/rtsig-max /proc/sys/kernel/ngroups_max /proc/sys/kernel/osrelease /proc/self/fd/%d/%s /proc/self/fd /proc/net 
"""
target_word = '/proc/self/exe'
verb = find_verb_of_vocab(sentence, target_word)
print(f'test3: {verb}')

doc = nlp(sentence)
svg = displacy.render(doc, style="dep", jupyter=True)

test3: None


In [12]:
sentence = """This Backdoor gathers the following data: CPU information Memory statistics IP address of infected machine 
Reads the following information from dir_proc: dir_proc_stat dir_proc_meminfo dir_proc_cpuinfo dir_proc_net dir_self"""
# dir_1 dir_2 dir_3 dir_4 dir_5 dir_6 -> 會把第一個 dir1 當作動詞

target_word = 'dir_self'
verb = find_verb_of_vocab(sentence, target_word)
print(f'test3: {verb}')

doc = nlp(sentence)
svg = displacy.render(doc, style="dep", jupyter=True)

test3: dir_1


### 客製化 tokenizer (Ver2)
1. 避免分割單字中的特殊字元
2. 將特殊檔案名稱改為名詞(NOUN)，而非標點符號(PUNCT)。經實驗 ruler 僅能改變詞性，對於 DP 的樹狀結構無改變，關聯的 dep_ 還是 branches -> punct。
3. 單獨將特殊檔案名稱 /proc/self/exe 改為 proc_self_exe，便可以成功同時改變詞性為 Noun 以及關聯為 reads -> dobj。做法是將

In [2]:
from pathlib import Path
import pandas as pd
from typing import Dict, List, get_type_hints
from IPython.display import display
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

In [3]:
# https://stackoverflow.com/questions/66636097/prevent-spacy-tokenizer-from-splitting-on-specific-character
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER, HYPHENS
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

nlp = spacy.load("en_core_web_sm")
text = "Get 10ct/liter off when using our App"
# Modify tokenizer infix patterns
infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        #r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
        r"(?<=[{a}0-9])[:<>=](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer
doc = nlp(text)
print('---- Origin nlp')
print([t.text for t in doc])
for token in doc:
    print('{:12}{:10}{:5}'.format(token.text, token.pos_, token.tag_))

# https://spacy.io/usage/linguistic-features#mappings-exceptions
# https://stackoverflow.com/questions/71418957/is-it-possible-to-to-add-a-part-of-speech-tag-to-a-spacy-tokenizer-special-case
from spacy.symbols import ORTH, POS, NOUN, VERB

ruler = nlp.get_pipe("attribute_ruler")
patterns = [[{"TEXT": "10ct/liter"}]]
verb_attrs = {"TAG": "NN", "POS": "NOUN"}
ruler.add(patterns=patterns, attrs=verb_attrs, index=0)

doc = nlp('Get 10ct/liter off when using our App')

print('---- After add ruler')
for token in doc:
    print('{:12}{:10}{:5}'.format(token.text, token.pos_, token.tag_))

ruler = nlp.get_pipe("attribute_ruler")
# patterns = [[{"TEXT": "/proc/self/exe"}]]
patterns = [[{"TEXT": "proc_self_exe"}]]
verb_attrs = {"TAG": "NN", "POS": "NOUN"} # verb_attrs = {"TAG": "NNP", "POS": "PROPN"}
# ruler.add(patterns=patterns, attrs=verb_attrs, index=0)

# doc = nlp('The main method then branches to function named get_executable_name which reads the symlink /proc/self/exe via readlink(..).')
doc = nlp('The main method then branches to function named get_executable_name which reads the symlink proc_self_exe via readlink(..).')

print('---- Real testcase')
for token in doc:
    print(f'{token.text: <14} {token.tag_:<4} {token.head.text:<10} {token.dep_}')
displacy.render(doc, style='dep', jupyter=True)

---- Origin nlp
['Get', '10ct/liter', 'off', 'when', 'using', 'our', 'App']
Get         VERB      VB   
10ct/liter  NUM       CD   
off         ADP       RP   
when        SCONJ     WRB  
using       VERB      VBG  
our         PRON      PRP$ 
App         PROPN     NNP  
---- After add ruler
Get         VERB      VB   
10ct/liter  NOUN      NN   
off         ADP       RP   
when        SCONJ     WRB  
using       VERB      VBG  
our         PRON      PRP$ 
App         PROPN     NNP  
---- Real testcase
The            DT   method     det
main           JJ   method     amod
method         NN   branches   nsubj
then           RB   branches   advmod
branches       NNS  branches   ROOT
to             TO   function   aux
function       VB   branches   xcomp
named          VBN  branches   acl
get_executable_name NNP  named      oprd
which          WDT  reads      nsubj
reads          VBZ  branches   relcl
the            DT   proc_self_exe det
symlink        NN   proc_self_exe compound
proc_se

In [7]:
svg = displacy.render(doc, style="dep", jupyter=False)
output_path = Path(f"./images/no10_read_symlink 修正後.svg") 
with output_path.open("w", encoding="utf-8") as f:
    f.write(svg)

In [8]:
doc = nlp("A string is formatted and the sed program is called which writes to the file in question")
svg = displacy.render(doc, style="dep", jupyter=False)
output_path = Path(f"./images/sed_is_called 句一.svg") 
with output_path.open("w", encoding="utf-8") as f:
    f.write(svg)

In [9]:
doc = nlp("(there are several string operations, such as sed -i -e '2 i%s/%s' /etc/rc.local is formatted for example).")
svg = displacy.render(doc, style="dep", jupyter=False)
output_path = Path(f"./images/sed_is_formatted 句二.svg") 
with output_path.open("w", encoding="utf-8") as f:
    f.write(svg)

### 寫成方法 (Ver2)

In [25]:
from typing import Callable, Dict, List, get_type_hints

def prepare_nlp() -> Callable:
    nlp = spacy.load("en_core_web_sm")
    text = "Get 10ct/liter off when using our App"
    # Modify tokenizer infix patterns
    infixes = (
        LIST_ELLIPSES
        + LIST_ICONS
        + [
            r"(?<=[0-9])[+\-\*^](?=[0-9-])",
            r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
                al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
            ),
            r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
            r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
            #r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
            r"(?<=[{a}0-9])[:<>=](?=[{a}])".format(a=ALPHA),
        ]
    )
    infix_re = compile_infix_regex(infixes)
    nlp.tokenizer.infix_finditer = infix_re.finditer

    return nlp

# a global funciton
customized_nlp = prepare_nlp()

def make_slash_to_underline(sentence: str, target_word: str) -> tuple[str, str]:
    ''' return sentence, target_word\n 
    將含有斜線 Special token 的置換成 token? 的單字，讓 Special token 能被識別成名詞(原本是punct)，也回傳置換過的 target_word '''
    ''' 將 Special token 中的斜線轉成底線，並去掉第一個斜線，試圖讓 spacy tagger & parser 讓 target_word 識別成名詞(原本是punct)。/proc/self/exe -> proc_self_exe '''
    # target_word = target_word.replace('/','_')
    # target_word = target_word.strip('_')
    # sentence = sentence.replace('/','_')
    # new_sent = [token.strip('_') for token in sentence.split()]
    # sentence = ' '.join(new_sent)
    new_sent = []
    cnt = 1
    for token in sentence.split():
        if '/' in token:
            new_sent.append(f"file_{cnt}")
            if target_word == token:
                target_word = f"file_{cnt}"
            cnt += 1
        else:
            new_sent.append(token)
    sentence = ' '.join(new_sent)
    return sentence, target_word

def find_verb_of_vocab(sentence: str, target_word: str, lemma=True):
    'Pass a sentence and a target_word, return the verb operates on the target_word or None. Lemmatization applied at default.'
    sentence, target_word = make_slash_to_underline(sentence, target_word)
    doc = customized_nlp(sentence)
    debug = False
    if sentence.startswith('(there are several'):
        displacy.render(doc, style='dep', jupyter=True)
        debug = True
    # find the target_word element (excat match)
    target_elem = None
    for i,token in enumerate(doc):
        if token.text == target_word:
            target_elem = token
    if target_elem is None:
        return None
    # find the verb parent of the target_word element
    token = target_elem
    while token.head != token:
        token = token.head
        if debug:
            print(token)
        # verb found
        if token.tag_.startswith('VB'):
            if token.lemma_ in ['m6_6n3', 'se', 'resolv.conf']: # verb refinement, add a black list of ev_verb. these words are not verb.
                continue 
            if lemma:
                return token.lemma_
            return token.text
    return None

# test 1: read
res = find_verb_of_vocab("The main method then branches to function named get_executable_name which reads the symlink /proc/self/exe via readlink(..). ", "/proc/self/exe")
print(res)

sent = """This Backdoor gathers the following data: 
CPU information Memory statistics IP address of infected machine 
Reads the following information from /proc: /proc/stat /proc/meminfo /proc/cpuinfo /proc/net/dev /proc/self/exe /proc/self/maps /proc/sys/vm/overcommit_memory /proc/sys/kernel/rtsig-max /proc/sys/kernel/ngroups_max /proc/sys/kernel/osrelease /proc/self/fd/%d/%s /proc/self/fd /proc/net"""
print(find_verb_of_vocab(sent, "/proc/self/exe")) # 'read'
print(find_verb_of_vocab(sent, "/proc/net/dev")) # 'read'
print(find_verb_of_vocab(sent, "/proc/stat")) # 'read'
print(find_verb_of_vocab("A string is formatted and the sed program is called which writes to the file in question", "sed")) # call
print(find_verb_of_vocab("(there are several string operations, such as sed -i -e '2 i%s/%s' /etc/rc.local is formatted for example).", "sed")) # format
sent = """This Backdoor adds the following processes:

sed -i -e '/exit/d' /etc/rc.local
sed -i -e '/^\r\n|\r|\n$/d' /etc/rc.local
sed -i -e '/%s/d' /etc/rc.local
sed -i -e '2 i%s/%s' /etc/rc.local
sed -i -e '2 i%s/%s start' /etc/rc.d/rc.local
sed -i -e '2 i%s/%s start' /etc/init.d/boot.local"""
print(find_verb_of_vocab(sent, "/etc/rc.local")) # 'add'
print(find_verb_of_vocab(sent, "/etc/init.d/boot.local")) # 'add'
sent = "Installation  This Backdoor adds the following processes:  sed -i -e '/exit/d' /etc/rc.local sed -i -e '/^\r\n|\r|\n$/d' /etc/rc.local sed -i -e '/%s/d' /etc/rc.local sed -i -e '2 i%s/%s' \
/etc/rc.local sed -i -e '2 i%s/%s start' /etc/rc.d/rc.local sed -i -e '2 i%s/%s start' /etc/init.d/boot.local Backdoor Routine "
print(find_verb_of_vocab(sent, "sed")) # 'add'


read
read
read
read
call


as
operations
are
be
add
add
add


In [11]:
'/' in 'proc_self_map'
'/' in 'proc/self/map'

True